In [2]:
import mysql.connector

In [12]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="test",
  database="insight_database"
)

In [13]:
cursor = mydb.cursor()

In [ ]:
cursor.execute("create table professors (id VARCHAR(255) NOT NULL, first_name VARCHAR(255) NOT NULL, last_name VARCHAR(255) NOT NULL, email VARCHAR(255), PRIMARY KEY(id));")

In [14]:
import pandas as pd

In [3]:
professors = pd.read_csv("../resources/professors.csv")

In [4]:
len(professors)

3401

In [5]:
len(professors.drop_duplicates(subset=["first_name", "last_name"]))

3401

In [15]:
sql = "INSERT into professors(id, first_name, last_name, email) values(%s, %s, %s, %s)"
for index, row in professors.iterrows():
    val = (row["id"], row["first_name"], row["last_name"], row["email"])
    cursor.execute(sql, val)

In [17]:
courses = pd.read_csv("courses.csv")

In [35]:
courses.fillna("None", inplace=True)

In [42]:
cursor.execute("drop table courses")

In [43]:
cursor.execute("create table courses (course_id VARCHAR(255), title VARCHAR(255) NOT NULL, class_level VARCHAR(255), PRIMARY KEY(course_id));")

In [45]:
sql = "INSERT into courses(course_id, title, class_level) values(%s, %s, %s)"
for index, row in courses.iterrows():
    course_id = row["subject_prefix"] + row["course_number"]
    val = (course_id, row["title"], row["class_level"])
    cursor.execute(sql, val)

In [15]:
cursor.execute("drop table grades")

In [16]:
cursor.execute("create table grades (professor_id VARCHAR(255) NOT NULL, course_id VARCHAR(255) NOT NULL, section_id VARCHAR(255) NOT NULL, A int, Aminus int, Bplus int, B int, Bminus int, Cplus int, C int, Cminus int, Dplus int, D int, Dminus int, F int, W int, PRIMARY KEY(professor_id, course_id, section_id), FOREIGN KEY (professor_id) References professors(id), FOREIGN KEY (course_id) references courses(course_id));")

In [17]:
grades = pd.read_csv("../resources/grades.csv")

In [18]:
sql = "insert into grades(professor_id, course_id, section_id, A, Aminus, Bplus, B, Bminus, Cplus, C, Cminus, Dplus, D, Dminus, F, W) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
i = 0
for index, row in grades.iterrows():
    if not pd.isna(row['section_id']):
        course_id = row["subject_prefix"] + row["course_number"]
        val = (row["professor_id"], course_id, row["section_id"], row["A"], row["A-"], row["B+"], row["B"], row["B-"], row["C+"], row["C"], row["C-"], row["D+"], row["D"], row["D-"], row["F"], row["W"])
        #print(row['section_id'])
        i+=1
        cursor.execute(sql, val)
print(i)

33191


In [8]:
cursor.close()

True

In [19]:
mydb.commit()
mydb.close()

In [7]:
grades = pd.read_csv("../resources/processed_course_grades.csv")

In [9]:
df = pd.merge(grades, professors, on=["first_name", "last_name"])

In [11]:
df.drop(["_id", "email", "phone_number"], axis=1, inplace=True)

In [36]:
df = df[['id'] + [col for col in df.columns if col != 'id'] ]

In [37]:
df.rename({'id':'professor_id'}, axis=1, inplace=True)

In [38]:
df.drop_duplicates(subset=["professor_id", "subject_prefix", "course_number", "section_id"], inplace=True)

In [39]:
df.dropna(inplace=True)

In [40]:
df.to_csv("../resources/grades.csv")